In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import numpy as np

def conv(z, h, omega_lambda, omega_matter):

    H0=100*h
#conversion Mpc --> km
    conv1=1./(1e6*206265*1.496e8)
#conversion de s --> Gyr
    conv2=3600*24*365.25*1e9
    H0=H0*conv1*conv2

    c1=np.sqrt(omega_lambda*(1.+z)**(-3.))+np.sqrt((omega_lambda*(1.+z)**(-3.))+omega_matter)
    ln=np.log(c1/np.sqrt(omega_matter))
    c2=2./(3.*np.sqrt(omega_lambda))
        
    time = (1./H0)*c2*ln

    return time


In [8]:
#Graficos de Materia Oscura

####PRUEBA PILOTO 2
import matplotlib.pyplot as plt
import h5py
import rotation as rot
import sphviewer as sph
import numpy as np

h=0.732
a0=1
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
vector = ('00','01','03','09','13','14','18','19','20','21')

snapshot=range(497,300,-1)
for isnap in snapshot:
    snap = h5py.File('/home/ornela/python/ornella/outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')

    a = snap['subhalo_000/Time'].value
    z = (a0/a)-1
    t = conv(z, h, omega_lambda, omega_matter)
    
    print isnap
    for i in range(0,1):
        pstr = snap['subhalo_0'+ str('%s'%vector[i]) +'/Str/Coordinates'].value
        cm   = snap['subhalo_0'+ str('%s'%vector[i]) +'/Center'].value
        vel  = snap['subhalo_0'+ str('%s'%vector[i]) + '/Str/Velocities'].value
        masa = snap['subhalo_0'+ str('%s'%vector[i]) +'/Str/Masses'].value
        r200 = snap['subhalo_0'+ str('%s' %vector[i])+ '/R200'].value
        
        pdrk = snap['subhalo_0'+ str('%s' %vector[i]) + '/Drk/Coordinates'].value
        masa_drk = snap['subhalo_0'+ str('%s' %vector[i])+ '/Drk/Masses'].value
        vel_drk  = snap['subhalo_0'+ str('%s'%vector[i]) + '/Drk/Velocities'].value
        
        
#---------paso las coordenadas respecto al centro de la galaxia-------
        xstr = (pstr[:,0]-cm[0])/h
        ystr = (pstr[:,1]-cm[1])/h
        zstr = (pstr[:,2]-cm[2])/h
        r = np.sqrt(xstr**2+ystr**2+zstr**2)
        
        xdrk = (pdrk[:,0]-cm[0])/h
        ydrk = (pdrk[:,1]-cm[1])/h
        zdrk = (pdrk[:,2]-cm[2])/h
        rdrk = np.sqrt(xdrk**2+ydrk**2+zdrk**2)
        
#---------------masas------------------------------------------        
        masa = masa/h
        
        rgal=0.15*r200
        #print r200
        
        limit = np.where(r<rgal)
        r_sort = np.sort(r[limit])
        r_indice = np.argsort(r[limit])
                
        Mc = np.cumsum((masa[limit])[r_indice])
        Mgal = Mc[-1]
               
        razon = Mc/Mgal
        noventa, = np.where(razon < 0.9)
        cincuenta, = np.where(razon < 0.5)
        
        r90 = r_sort[noventa]
        r50 = r_sort[cincuenta]
#------------------------------------------------------------------        
##esto sirve si quiero hacer una submuestra de mi conjunto de datos
       
        veloc,=np.where(r<r50[-1])
        
#----------componentes de la velocidad del centro de masa------------
        vxcm = sum(masa[veloc]*vel[veloc,0])/sum(masa[veloc])
        vycm = sum(masa[veloc]*vel[veloc,1])/sum(masa[veloc])
        vzcm = sum(masa[veloc]*vel[veloc,2])/sum(masa[veloc])
        
        vxcm_drk = sum(masa_drk[veloc]*vel_drk[veloc,0])/sum(masa_drk[veloc])
        vycm_drk = sum(masa_drk[veloc]*vel_drk[veloc,1])/sum(masa_drk[veloc])
        vzcm_drk = sum(masa_drk[veloc]*vel_drk[veloc,2])/sum(masa_drk[veloc])
#----- velocidades de las estrellas respecto del centro de masa de la galaxia---------
        vx = vel[:,0] - vxcm
        vy = vel[:,1] - vycm
        vz = vel[:,2] - vzcm
        
        vx_drk = vel_drk[:,0] - vxcm_drk
        vy_drk = vel_drk[:,1] - vycm_drk
        vz_drk = vel_drk[:,2] - vzcm_drk
# utilizo una matriz de rotacion de modo que el plano xy coincida aprox con el plano del disco
# rot.rot, el primer rot es como yo le llamo arriba cuando importo, el segundo rot tiene que ver
#con como se llama la funcion adentro del archivo rotation
        if isnap == 497: 
            e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(masa,xstr,ystr,zstr,vx,vy,vz,r90[-1])

##posiciones de partículas que se quiere graficar
##como lo de arriba me da los versores hago las posiciones con esto 

        xn = e1x*xdrk + e1y*ydrk + e1z*zdrk
        yn = e2x*xdrk + e2y*ydrk + e2z*zdrk
        zn = e3x*xdrk + e3y*ydrk + e3z*zdrk
        vxn = e1x*vx_drk + e1y*vy_drk + e1z*vz_drk
        vyn = e2x*vx_drk + e2y*vy_drk + e2z*vz_drk
        vzn = e3x*vx_drk + e3y*vy_drk + e3z*vz_drk

        pos=np.ndarray([3,np.size(xn)])
        pos[0,:]=xn
        pos[1,:]=yn
        pos[2,:]=zn

#---------------------generador del gráfico-----------------
        rl=r200    
        corte,=np.where((xn<rl) & (yn<rl) & (zn<rl) & (xn>-rl) & (yn>-rl) & (zn>-rl))
         
#-----rango que tiene la escala  de colores-----
        vmin=0
        vmax=9
        
# ----escala de colores que te guste (http://matplotlib.org/examples/color/colormaps_reference.html)---
        cmap='nipy_spectral'
    
#-----grafico-----------
        fig=plt.figure(1,figsize=(8,8))
        fig.subplots_adjust(bottom=0.08, left =0.09, right = 0.95, top = 0.95, wspace=0.3, hspace= 0.05)
        
        particles=sph.Particles(pos[:,corte],masa_drk[corte]*1e10,nb=30)
        escena=sph.Scene(particles)
        escena.update_camera(r='infinity',x=0,y=0,z=0,extent=[-rl,rl,-rl,rl])
        rend=sph.Render(escena)
        extent=escena.get_extent()
        rend.set_logscale()
        
        ax1=fig.add_subplot(111)
        plt.imshow(rend.get_image(),extent=extent,origin='lower',cmap=cmap)
        plt.axis('off')
        #plt.xlabel('$x$'+' $[kpc]$', fontsize=26)
        #plt.ylabel('$y$'+' $[kpc]$', fontsize=26)
        #ax1.minorticks_on()
        #ax1.tick_params( labelsize=20)
        #ax1.tick_params('both', length=5, width=1.8,which='minor')
        #ax1.tick_params('both', length=8, width=1.8,which='major')
        plt.text(-100, 100, str('%.3f'%t)+'Gyr', fontsize=22, color='white', ha='left', va='center') 
                 #bbox={'facecolor':'black', 'alpha':1, 'pad':2})
        plt.annotate("",xy=(50, -100), xycoords='data',xytext=(100, -100),textcoords='data',
                     ha='center', va='center', 
                     arrowprops=dict(arrowstyle="|-|", connectionstyle='arc3', color ='white', lw=1.5))
        
        plt.text(75, -100, '50 kpc', fontsize=15, color='white', ha='center', va='bottom')
        
        
        fig.savefig('/home/ornela/python/sem1/DM_M31/'+str('%03d'%isnap)+'-sh0'+str('%s'%vector[i])+'.png', facecolor= 'k', edgecolor='k', frameon= 'False',bbox_inches='tight',pad_inches=0, dpi = 100)
        #fig.savefig('/home/ornela/python/sem1/star3_'+str('%s'%vector[i])+'.png', dpi = 100)
        #plt.show()
        plt.clf()

497
496
495
494
493
492
491
490
489
488
487
486
485
484
483
482
481
480
479
478
477
476
475
474
473
472
471
470
469
468
467
466
465
464
463
462
461
460
459
458
457
456
455
454
453
452
451
450
449
448
447
446
445
444
443
442
441
440
439
438
437
436
435
434
433
432
431
430
429
428
427
426
425
424
423
422
421
420
419
418
417
416
415
414
413
412
411
410
409
408
407
406
405
404
403
402
401
400
399
398
397
396
395
394
393
392
391
390
389
388
387
386
385
384
383
382
381
380
379
378
377
376
375
374
373
372
371
370
369
368
367
366
365
364
363
362
361
360
359
358
357
356
355
354
353
352
351
350
349
348
347
346
345
344
343
342
341
340
339
338
337
336
335
334
333
332
331
330
329
328
327
326
325
324
323
322
321
320
319
318
317
316
315
314
313
312
311
310
309
308
307
306
305
304
303
302
301
